# Houseprice prediction Ensemble Model with LIME Explainer Corrected Correlation Pipelined 


In [1]:
# Import libraries
import sklearn
from scipy.stats import chi2_contingency
import numpy as np
import pandas as pd
from sklearn_pandas import DataFrameMapper
import matplotlib.pyplot as plt
from  matplotlib.colors import LinearSegmentedColormap
gray_cmap=LinearSegmentedColormap.from_list('gy',[(.3,.3,.3),(.8,.8,.8)], N=2) 
np.random.seed(222) #Seed for random numbers generation
import seaborn as sns
%matplotlib inline
import os

import lime
import lime.lime_tabular
import lime.explanation

from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import LabelEncoder

sns.set_style('darkgrid')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 90)

from sklearn.neighbors import KNeighborsRegressor
import scipy.stats
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.preprocessing import FunctionTransformer
from pycaret.regression import setup, compare_models
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import *

from catboost import CatBoostRegressor
from sklearn.linear_model import BayesianRidge, OrthogonalMatchingPursuit
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, ExtraTreesRegressor
from xgboost import XGBRegressor

def set_plot_style():
  plt.axis([-2,2,-2,2])
  plt.xlabel('x1')
  plt.ylabel('x2')


## Data Import

In [2]:
train0 = pd.read_csv('data/house_prices/train.csv')
test0 = pd.read_csv('data/house_prices/test.csv')

In [3]:
# Split original training set into train and validation sets
train_set, val_set = train_test_split(train0, test_size=0.2, random_state=42)

In [4]:
# Separate features and targets for train and validation sets
X_train = train_set.drop(['Id', 'SalePrice'], axis=1)
y_train = train_set['SalePrice']

X_val = val_set.drop(['Id', 'SalePrice'], axis=1)
y_val = val_set['SalePrice']

# Separate features for test set
X_test = test0.drop('Id', axis=1)

## Preprocessing:

### Cleaning

In [5]:
# Define columns
numeric_cols = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = X_train.select_dtypes(include=['object']).columns.tolist()

In [6]:
# Pipeline 1: Imputation
numeric_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=5))])

categorical_transformer= Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent'))])

preprocessor1 = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)])

# Fit and Transform
preprocessor1.fit(X_train)
X_train_imputed = pd.DataFrame(preprocessor1.transform(X_train), columns=numeric_cols + categorical_cols)
X_val_imputed = pd.DataFrame(preprocessor1.transform(X_val), columns=numeric_cols + categorical_cols)
X_test_imputed = pd.DataFrame(preprocessor1.transform(X_test), columns=numeric_cols + categorical_cols)

# Cast numeric columns back to their original types
X_train_imputed[numeric_cols] = X_train_imputed[numeric_cols].apply(pd.to_numeric, errors='coerce')
X_val_imputed[numeric_cols] = X_val_imputed[numeric_cols].apply(pd.to_numeric, errors='coerce')
X_test_imputed[numeric_cols] = X_test_imputed[numeric_cols].apply(pd.to_numeric, errors='coerce')

In [7]:
X_train_imputed

MSSubClass  LotFrontage  LotArea  OverallQual  OverallCond  YearBuilt  \
0           20.0         70.0   8400.0          5.0          6.0     1957.0   
1           60.0         59.0   7837.0          6.0          7.0     1993.0   
2           30.0         67.0   8777.0          5.0          7.0     1910.0   
3           50.0         60.0   7200.0          5.0          7.0     1937.0   
4           50.0         50.0   5000.0          5.0          6.0     1924.0   
...          ...          ...      ...          ...          ...        ...   
1163        20.0         78.0   9317.0          6.0          5.0     2006.0   
1164        50.0         65.0   7804.0          4.0          3.0     1928.0   
1165        20.0         60.0   8172.0          5.0          7.0     1955.0   
1166        50.0         55.0   7642.0          7.0          8.0     1918.0   
1167       120.0         53.0   3684.0          7.0          5.0     2007.0   

      YearRemodAdd  MasVnrArea  BsmtFinSF1  BsmtFinSF2  BsmtUnfSF  \
0           1957.0         0.0       922.0         0.0      392.0   
1           1994.0         0.0         0.0         0.0      799.0   
2           1950.0         0.0         0.0         0.0      796.0   
3           1950.0       252.0       569.0         0.0      162.0   
4           1950.0         0.0       218.0         0.0      808.0   
...            ...         ...         ...         ...        ...   
1163        2006.0         0.0        24.0         0.0     1290.0   
1164        1950.0         0.0       622.0         0.0      500.0   
1165        1990.0         0.0       167.0         0.0      697.0   
1166        1998.0         0.0         0.0         0.0      912.0   
1167        2007.0       130.0         0.0         0.0     1373.0   

      TotalBsmtSF  1stFlrSF  2ndFlrSF  LowQualFinSF  GrLivArea  BsmtFullBath  \
0          1314.0    1314.0       0.0           0.0     1314.0           1.0   
1           799.0     799.0     772.0           0.0     1571.0           0.0   
2           796.0     796.0       0.0           0.0      796.0           0.0   
3           731.0     981.0     787.0           0.0     1768.0           1.0   
4          1026.0    1026.0     665.0           0.0     1691.0           0.0   
...           ...       ...       ...           ...        ...           ...   
1163       1314.0    1314.0       0.0           0.0     1314.0           0.0   
1164       1122.0    1328.0     653.0           0.0     1981.0           1.0   
1165        864.0     864.0       0.0           0.0      864.0           1.0   
1166        912.0     912.0     514.0           0.0     1426.0           0.0   
1167       1373.0    1555.0       0.0           0.0     1555.0           0.0   

      BsmtHalfBath  FullBath  HalfBath  BedroomAbvGr  KitchenAbvGr  \
0              0.0       1.0       0.0           3.0           1.0   
1              0.0       2.0       1.0           3.0           1.0   
2              0.0       1.0       0.0           2.0           1.0   
3              0.0       1.0       1.0           3.0           1.0   
4              0.0       2.0       0.0           3.0           1.0   
...            ...       ...       ...           ...           ...   
1163           0.0       2.0       0.0           3.0           1.0   
1164           0.0       2.0       0.0           4.0           1.0   
1165           0.0       1.0       0.0           2.0           1.0   
1166           0.0       1.0       1.0           3.0           1.0   
1167           0.0       2.0       0.0           2.0           1.0   

      TotRmsAbvGrd  Fireplaces  GarageYrBlt  GarageCars  GarageArea  \
0              5.0         0.0       1957.0         1.0       294.0   
1              7.0         1.0       1993.0         2.0       380.0   
2              4.0         0.0       1948.0         0.0         0.0   
3              7.0         2.0       1939.0         1.0       240.0   
4              6.0         1.0       1924.0         1.0       308.0   


### Correlation Correction

In [8]:
# Pipeline 2: Correlation Correction

def cramers_v(x, y):
    confusion_matrix = pd.crosstab(x, y)
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))


class RemoveCorrelatedFeatures(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=0.7, categorical_threshold=0.7):
        self.threshold = threshold
        self.categorical_threshold = categorical_threshold

    def fit(self, X, y=None):
        self.correlated_features = set()
        self.correlated_categorical_features = set()
        
        # For numerical features
        correlation_matrix = X.corr()

        for i in range(len(correlation_matrix.columns)):
            for j in range(i):
                if abs(correlation_matrix.iloc[i, j]) > self.threshold:
                    colname = correlation_matrix.columns[i]
                    self.correlated_features.add(colname)
        
        # For categorical features
        cat_cols = X.select_dtypes(include=['object']).columns.tolist()
        for i, col1 in enumerate(cat_cols):
            for j, col2 in enumerate(cat_cols[i+1:]):
                if cramers_v(X[col1], X[col2]) > self.categorical_threshold:
                    self.correlated_categorical_features.add(col2)
        
        return self

    def transform(self, X, y=None):
        X = X.drop(columns=self.correlated_features)
        X = X.drop(columns=self.correlated_categorical_features)
        return X

preprocessor2 = Pipeline(steps=[
    ('remove_correlated_features', RemoveCorrelatedFeatures(threshold=0.7, categorical_threshold=0.6))
])

preprocessor2.fit(X_train_imputed)
X_train_corrected = pd.DataFrame(preprocessor2.transform(X_train_imputed))
X_val_corrected = pd.DataFrame(preprocessor2.transform(X_val_imputed))
X_test_corrected = pd.DataFrame(preprocessor2.transform(X_test_imputed))

In [9]:
X_train_corrected

MSSubClass  LotFrontage  LotArea  OverallQual  OverallCond  YearBuilt  \
0           20.0         70.0   8400.0          5.0          6.0     1957.0   
1           60.0         59.0   7837.0          6.0          7.0     1993.0   
2           30.0         67.0   8777.0          5.0          7.0     1910.0   
3           50.0         60.0   7200.0          5.0          7.0     1937.0   
4           50.0         50.0   5000.0          5.0          6.0     1924.0   
...          ...          ...      ...          ...          ...        ...   
1163        20.0         78.0   9317.0          6.0          5.0     2006.0   
1164        50.0         65.0   7804.0          4.0          3.0     1928.0   
1165        20.0         60.0   8172.0          5.0          7.0     1955.0   
1166        50.0         55.0   7642.0          7.0          8.0     1918.0   
1167       120.0         53.0   3684.0          7.0          5.0     2007.0   

      YearRemodAdd  MasVnrArea  BsmtFinSF1  BsmtFinSF2  BsmtUnfSF  \
0           1957.0         0.0       922.0         0.0      392.0   
1           1994.0         0.0         0.0         0.0      799.0   
2           1950.0         0.0         0.0         0.0      796.0   
3           1950.0       252.0       569.0         0.0      162.0   
4           1950.0         0.0       218.0         0.0      808.0   
...            ...         ...         ...         ...        ...   
1163        2006.0         0.0        24.0         0.0     1290.0   
1164        1950.0         0.0       622.0         0.0      500.0   
1165        1990.0         0.0       167.0         0.0      697.0   
1166        1998.0         0.0         0.0         0.0      912.0   
1167        2007.0       130.0         0.0         0.0     1373.0   

      TotalBsmtSF  2ndFlrSF  LowQualFinSF  GrLivArea  BsmtFullBath  \
0          1314.0       0.0           0.0     1314.0           1.0   
1           799.0     772.0           0.0     1571.0           0.0   
2           796.0       0.0           0.0      796.0           0.0   
3           731.0     787.0           0.0     1768.0           1.0   
4          1026.0     665.0           0.0     1691.0           0.0   
...           ...       ...           ...        ...           ...   
1163       1314.0       0.0           0.0     1314.0           0.0   
1164       1122.0     653.0           0.0     1981.0           1.0   
1165        864.0       0.0           0.0      864.0           1.0   
1166        912.0     514.0           0.0     1426.0           0.0   
1167       1373.0       0.0           0.0     1555.0           0.0   

      BsmtHalfBath  FullBath  HalfBath  BedroomAbvGr  KitchenAbvGr  \
0              0.0       1.0       0.0           3.0           1.0   
1              0.0       2.0       1.0           3.0           1.0   
2              0.0       1.0       0.0           2.0           1.0   
3              0.0       1.0       1.0           3.0           1.0   
4              0.0       2.0       0.0           3.0           1.0   
...            ...       ...       ...           ...           ...   
1163           0.0       2.0       0.0           3.0           1.0   
1164           0.0       2.0       0.0           4.0           1.0   
1165           0.0       1.0       0.0           2.0           1.0   
1166           0.0       1.0       1.0           3.0           1.0   
1167           0.0       2.0       0.0           2.0           1.0   

      Fireplaces  GarageCars  WoodDeckSF  OpenPorchSF  EnclosedPorch  \
0            0.0         1.0       250.0          0.0            0.0   
1            1.0         2.0         0.0         40.0            0.0   
2            0.0         0.0       328.0          0.0          164.0   
3            2.0         1.0         0.0          0.0          264.0   
4            1.0         1.0         0.0          0.0          242.0   
...          ...         ...         ...          ...            ...   
1163         1.0         2.0         0.0  

### Feature Normalization

In [10]:
# Pipeline 3: Numerical Feature Normalization
class LogTransformSkewedFeatures(BaseEstimator, TransformerMixin):
    def __init__(self, skew_threshold=0.5):
        self.skew_threshold = skew_threshold

    def fit(self, X, y=None):
        numeric_data = X.select_dtypes(np.number)
        skew_df = pd.DataFrame(numeric_data.columns, columns=['Feature'])
        skew_df['Skew'] = skew_df['Feature'].apply(lambda feature: scipy.stats.skew(X[feature]))
        skew_df['Absolute Skew'] = skew_df['Skew'].apply(abs)
        skew_df['Skewed'] = skew_df['Absolute Skew'].apply(lambda x: True if x >= self.skew_threshold else False)
        self.skewed_features = skew_df.query("Skewed == True")['Feature'].values
        return self

    def transform(self, X, y=None):
        X_transformed = X.copy()
        for column in self.skewed_features:
            X_transformed[column] = np.log1p(X_transformed[column])
        return X_transformed

preprocessor3 = Pipeline(steps=[
    ('log_transform_skewed_features', LogTransformSkewedFeatures(skew_threshold=0.5))
])

# Fit the pipeline to the training data
preprocessor3.fit(X_train_corrected)

# Transform all data sets
X_train_normalized = pd.DataFrame(preprocessor3.transform(X_train_corrected))
X_val_normalized = pd.DataFrame(preprocessor3.transform(X_val_corrected))
X_test_normalized = pd.DataFrame(preprocessor3.transform(X_test_corrected))


In [11]:
X_train_normalized

MSSubClass  LotFrontage   LotArea  OverallQual  OverallCond  YearBuilt  \
0       3.044522     4.262680  9.036106          5.0     1.945910   7.579679   
1       4.110874     4.094345  8.966739          6.0     2.079442   7.597898   
2       3.433987     4.219508  9.080004          5.0     2.079442   7.555382   
3       3.931826     4.110874  8.881975          5.0     2.079442   7.569412   
4       3.931826     3.931826  8.517393          5.0     1.945910   7.562681   
...          ...          ...       ...          ...          ...        ...   
1163    3.044522     4.369448  9.139703          6.0     1.791759   7.604396   
1164    3.931826     4.189655  8.962520          4.0     1.386294   7.564757   
1165    3.044522     4.110874  9.008591          5.0     2.079442   7.578657   
1166    3.931826     4.025352  8.941545          7.0     2.197225   7.559559   
1167    4.795791     3.988984  8.212026          7.0     1.791759   7.604894   

      YearRemodAdd  MasVnrArea  BsmtFinSF1  BsmtFinSF2  BsmtUnfSF  \
0         7.579679    0.000000    6.827629         0.0   5.973810   
1         7.598399    0.000000    0.000000         0.0   6.684612   
2         7.576097    0.000000    0.000000         0.0   6.680855   
3         7.576097    5.533389    6.345636         0.0   5.093750   
4         7.576097    0.000000    5.389072         0.0   6.695799   
...            ...         ...         ...         ...        ...   
1163      7.604396    0.000000    3.218876         0.0   7.163172   
1164      7.576097    0.000000    6.434547         0.0   6.216606   
1165      7.596392    0.000000    5.123964         0.0   6.548219   
1166      7.600402    0.000000    0.000000         0.0   6.816736   
1167      7.604894    4.875197    0.000000         0.0   7.225481   

      TotalBsmtSF  2ndFlrSF  LowQualFinSF  GrLivArea  BsmtFullBath  \
0        7.181592  0.000000           0.0   7.181592      0.693147   
1        6.684612  6.650279           0.0   7.360104      0.000000   
2        6.680855  0.000000           0.0   6.680855      0.000000   
3        6.595781  6.669498           0.0   7.478170      0.693147   
4        6.934397  6.501290           0.0   7.433667      0.000000   
...           ...       ...           ...        ...           ...   
1163     7.181592  0.000000           0.0   7.181592      0.000000   
1164     7.023759  6.483107           0.0   7.591862      0.693147   
1165     6.762730  0.000000           0.0   6.762730      0.693147   
1166     6.816736  6.244167           0.0   7.263330      0.000000   
1167     7.225481  0.000000           0.0   7.349874      0.000000   

      BsmtHalfBath  FullBath  HalfBath  BedroomAbvGr  KitchenAbvGr  \
0              0.0       1.0  0.000000           3.0      0.693147   
1              0.0       2.0  0.693147           3.0      0.693147   
2              0.0       1.0  0.000000           2.0      0.693147   
3              0.0       1.0  0.693147           3.0      0.693147   
4              0.0       2.0  0.000000           3.0      0.693147   
...            ...       ...       ...           ...           ...   
1163           0.0       2.0  0.000000           3.0      0.693147   
1164           0.0       2.0  0.000000           4.0      0.693147   
1165           0.0       1.0  0.000000           2.0      0.693147   
1166           0.0       1.0  0.693147           3.0      0.693147   
1167           0.0       2.0  0.000000           2.0      0.693147   

      Fireplaces  GarageCars  WoodDeckSF  OpenPorchSF  EnclosedPorch  \
0       0.000000         1.0    5.525453     0.000000       0.000000   
1       0.693147         2.0    0.000000     3.713572       0.000000   
2       0.000000         0.0    5.796058     0.000000       5.105945   
3       1.098612         1.0    0.000000     0.000000       5.579730   
4       0.693147         1.0    0.000000     0.000000       5.493061   
...          ...         ...         ...          ...            ...   
1163    0.693147         2.0  

### Encoding & Scaling:

In [12]:
# Pipeline 4: Encode Categoricals & Scale Numericals
categorical_columns = X_train_corrected.select_dtypes(include=['object']).columns.tolist()
numerical_columns = X_train_corrected.select_dtypes(include=['float64']).columns.tolist()

# Custom Transformer to handle LabelEncoding for multiple columns
class CustomLabelEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, default_label="UNKNOWN"):
        self.label_encoders = {}
        self.default_label = default_label
        
    def fit(self, X, y=None):
        for col in X.columns:
            le = LabelEncoder()
            # Add a default category for unseen labels
            unique_values = pd.Series([self.default_label]).append(X[col])
            le.fit(unique_values)
            self.label_encoders[col] = le
        return self
    
    def transform(self, X):
        X_encoded = X.copy()
        for col in X.columns:
            le = self.label_encoders[col]
            # Replace unseen labels with the default label
            X_encoded[col] = X[col].apply(lambda x: x if x in le.classes_ else self.default_label)
            X_encoded[col] = le.transform(X_encoded[col])
        return X_encoded

# Label encoder pipeline for categorical features
categorical_transformer = Pipeline(steps=[
    ('labelencoder', CustomLabelEncoder(default_label='UNKNOWN'))
])

# Standard scaler for numerical features
numerical_transformer = Pipeline(steps=[
    ('identity', FunctionTransformer())
])

# Column transformer
preprocessor4 = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_columns),
        ('cat', categorical_transformer, categorical_columns)
    ])

# Fitting the pipeline to your training data
preprocessor4.fit(X_train_corrected)

# Transforming the datasets
X_train_final = preprocessor4.transform(X_train_corrected)
X_val_final = preprocessor4.transform(X_val_corrected)
X_test_final = preprocessor4.transform(X_test_corrected)


# Combine the two lists of feature names in the order that the ColumnTransformer uses them
combined_feature_names = numerical_columns + categorical_columns

# Converting them back to DataFrames
X_train_final = pd.DataFrame(X_train_final, columns=combined_feature_names)
X_val_final = pd.DataFrame(X_val_final, columns=combined_feature_names)
X_test_final = pd.DataFrame(X_test_final, columns=combined_feature_names)

In [13]:
X_train_final

MSSubClass  LotFrontage  LotArea  OverallQual  OverallCond  YearBuilt  \
0           20.0         70.0   8400.0          5.0          6.0     1957.0   
1           60.0         59.0   7837.0          6.0          7.0     1993.0   
2           30.0         67.0   8777.0          5.0          7.0     1910.0   
3           50.0         60.0   7200.0          5.0          7.0     1937.0   
4           50.0         50.0   5000.0          5.0          6.0     1924.0   
...          ...          ...      ...          ...          ...        ...   
1163        20.0         78.0   9317.0          6.0          5.0     2006.0   
1164        50.0         65.0   7804.0          4.0          3.0     1928.0   
1165        20.0         60.0   8172.0          5.0          7.0     1955.0   
1166        50.0         55.0   7642.0          7.0          8.0     1918.0   
1167       120.0         53.0   3684.0          7.0          5.0     2007.0   

      YearRemodAdd  MasVnrArea  BsmtFinSF1  BsmtFinSF2  BsmtUnfSF  \
0           1957.0         0.0       922.0         0.0      392.0   
1           1994.0         0.0         0.0         0.0      799.0   
2           1950.0         0.0         0.0         0.0      796.0   
3           1950.0       252.0       569.0         0.0      162.0   
4           1950.0         0.0       218.0         0.0      808.0   
...            ...         ...         ...         ...        ...   
1163        2006.0         0.0        24.0         0.0     1290.0   
1164        1950.0         0.0       622.0         0.0      500.0   
1165        1990.0         0.0       167.0         0.0      697.0   
1166        1998.0         0.0         0.0         0.0      912.0   
1167        2007.0       130.0         0.0         0.0     1373.0   

      TotalBsmtSF  2ndFlrSF  LowQualFinSF  GrLivArea  BsmtFullBath  \
0          1314.0       0.0           0.0     1314.0           1.0   
1           799.0     772.0           0.0     1571.0           0.0   
2           796.0       0.0           0.0      796.0           0.0   
3           731.0     787.0           0.0     1768.0           1.0   
4          1026.0     665.0           0.0     1691.0           0.0   
...           ...       ...           ...        ...           ...   
1163       1314.0       0.0           0.0     1314.0           0.0   
1164       1122.0     653.0           0.0     1981.0           1.0   
1165        864.0       0.0           0.0      864.0           1.0   
1166        912.0     514.0           0.0     1426.0           0.0   
1167       1373.0       0.0           0.0     1555.0           0.0   

      BsmtHalfBath  FullBath  HalfBath  BedroomAbvGr  KitchenAbvGr  \
0              0.0       1.0       0.0           3.0           1.0   
1              0.0       2.0       1.0           3.0           1.0   
2              0.0       1.0       0.0           2.0           1.0   
3              0.0       1.0       1.0           3.0           1.0   
4              0.0       2.0       0.0           3.0           1.0   
...            ...       ...       ...           ...           ...   
1163           0.0       2.0       0.0           3.0           1.0   
1164           0.0       2.0       0.0           4.0           1.0   
1165           0.0       1.0       0.0           2.0           1.0   
1166           0.0       1.0       1.0           3.0           1.0   
1167           0.0       2.0       0.0           2.0           1.0   

      Fireplaces  GarageCars  WoodDeckSF  OpenPorchSF  EnclosedPorch  \
0            0.0         1.0       250.0          0.0            0.0   
1            1.0         2.0         0.0         40.0            0.0   
2            0.0         0.0       328.0          0.0          164.0   
3            2.0         1.0         0.0          0.0          264.0   
4            1.0         1.0         0.0          0.0          242.0   
...          ...         ...         ...          ...            ...   
1163         1.0         2.0         0.0  

### Target Transformation

##### Applying the log on the target:

In [14]:
# Apply log transformation to the target variable
y_train_log = np.log1p(y_train)  # using log1p to also deal with any zeros
y_val_log = np.log1p(y_val)

In [15]:
y_train_log

254     11.884496
1066    12.089544
638     11.350418
799     12.072547
380     11.751950
          ...    
1095    12.080696
1130    11.813037
1294    11.652696
860     12.154521
1126    12.066816
Name: SalePrice, Length: 1168, dtype: float64

## Creating the models:

### Catboost Regressor

In [16]:
catboost = CatBoostRegressor(verbose=0)
catboost.fit(X_train_final, y_train)

catboost_predict = catboost.predict(X_test_final)


### Bayesian Ridge

In [17]:
br = BayesianRidge()
br.fit(X_train_final, y_train)

br_predict = br.predict(X_test_final)

### OrthogonalMatchingPursuit

In [18]:
omp = OrthogonalMatchingPursuit()
omp.fit(X_train_final, y_train)

omp_predict = omp.predict(X_test_final)

### Gradient Boosting Regressor

In [19]:
gbr = GradientBoostingRegressor()
gbr.fit(X_train_final, y_train)

kf = KFold(n_splits=10)
result = np.exp(np.sqrt(-cross_val_score(gbr, X_train_final, y_train, scoring='neg_mean_squared_error', cv=kf)))

gbr_best = result
gbr_best

gbr_predict = np.exp(gbr.predict(X_test_final))

### XGBoost Regressor

In [20]:
xgboost = XGBRegressor()
xgboost.fit(X_train_final, y_train)

xgboost_predict = xgboost.predict(X_test_final)

In [21]:
## Ensemble with VotingRegressor

In [22]:
from sklearn.ensemble import VotingRegressor

In [23]:
estimators = [(catboost), (br), (omp), (gbr), (xgboost)]

In [24]:
ensemble = VotingRegressor([('catboost', catboost), ('br', br), ('omp', omp), ('gbr', gbr), ('xgboost', xgboost)], weights = [0.4, 0.2, 0.1, 0.2, 0.1])

In [25]:
ensemble.fit(X_train_final, y_train)

VotingRegressor(estimators=[('catboost',
                             <catboost.core.CatBoostRegressor object at 0x7fa621a82d30>),
                            ('br', BayesianRidge()),
                            ('omp', OrthogonalMatchingPursuit()),
                            ('gbr', GradientBoostingRegressor()),
                            ('xgboost',
                             XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None,
                                          early_stopping_rounds=N...
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=None, max_bin=None,
                                          max_cat_threshold=None,
                                          max_cat_to_onehot=None,
                                          max_delta_step=None, max_depth=None,
                                          max_leaves=None,
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
                                          n_estimators=100, n_jobs=None,
                                          num_parallel_tree=None,
                                          predictor=None, random_state=None, ...))],
                weights=[0.4, 0.2, 0.1, 0.2, 0.1])

In [26]:
final_predictions = ensemble.predict(X_test_final)

## Importing true SalePrice values

In [27]:
true_prices = pd.read_csv('data/house_prices/solution.csv')


In [28]:
y_true = true_prices['SalePrice'].values

## Metrics

In [29]:
print('Mean absolute error: ' + str(mean_absolute_error(y_true,final_predictions)))
print('Mean squared error: ' + str(mean_squared_error(y_true,final_predictions)))
print('Root mean squared error: ' + str(np.sqrt(mean_squared_error(y_true,final_predictions))))

Mean absolute error: 14190.016257205
Mean squared error: 518902294.6653912
Root mean squared error: 22779.427004764435


In [30]:
print('Explained variance score: ' + str(explained_variance_score(y_true,final_predictions)))

Explained variance score: 0.9204566957113982


In [31]:

print('R2 score: ' + str(r2_score(y_true,final_predictions)))

R2 score: 0.9199142174922641


###### -> R2 score of 0.919 indicates good model quality

## LIME Explainer

### Handling categorical features for LIME

In [32]:
# Identify which columns are categorical in the resulting DataFrame
categorical_feature_names = categorical_columns  # you've already had this in your code
categorical_indices = [X_train_final.columns.get_loc(c) for c in categorical_feature_names if c in X_train_final]

# Check if the data in these columns are all integers
for index in categorical_indices:
    column_name = X_train_final.columns[index]
    if not np.all(X_train_final[column_name].astype(int) == X_train_final[column_name]):
        raise ValueError(f"Column {column_name} contains non-integer values")

# Now, categorical_indices contains the integer indices of the categorical columns, and they're verified to contain only integers
categorical_features = categorical_indices

In [33]:
# Initialize an empty dictionary to hold the mapping
categorical_names = {}

# Iterate over the categorical columns and their corresponding indices
for index, col in enumerate(categorical_columns):
    # Retrieve the LabelEncoder instance used for this column
    le = preprocessor4.named_transformers_['cat'].named_steps['labelencoder'].label_encoders[col]
    
    # Retrieve the classes (i.e., the unique labels)
    classes = le.classes_
    
    # Find the index of the column in the transformed DataFrame
    transformed_index = X_train_final.columns.get_loc(col)
    
    # Add to the dictionary
    categorical_names[transformed_index] = classes

In [34]:
def ensemble_predict(data):

    predictions = ensemble.predict(data)
    
    return predictions

In [35]:
explainer = lime.lime_tabular.LimeTabularExplainer(
    training_data = np.array(X_train_final),
    feature_names = X_train_final.columns,
    categorical_features=categorical_features,
    categorical_names=categorical_names,
    verbose = True, mode = 'regression')

In [36]:
def predict_fn_for_lime(instance):
    prediction = ensemble_predict(instance)
    return prediction

In [37]:
np.random.seed(1)
i = 3
original_row = X_test_corrected.iloc[i]
expl = explainer.explain_instance(
    data_row = X_test_final.iloc[i], 
    predict_fn = predict_fn_for_lime, 
    num_features = 5
)

expl.show_in_notebook(show_all=False)

# Save the LIME explanation to an HTML file
output_dir = 'figures/lime'
html_path = os.path.join(output_dir, "lime_output.html")
expl.save_to_file(html_path)


Intercept 173874.8459109568
Prediction_local [175734.49455065]
Right: 191600.79459261912


### LIME Output with custom kernel_width

In [38]:
explainer = lime.lime_tabular.LimeTabularExplainer(
    training_data = np.array(X_train_final),
    feature_names = X_train_final.columns,
    categorical_features=categorical_features,
    categorical_names=categorical_names,
    kernel_width= 0.8,
    verbose = True, mode = 'regression')

In [39]:
def predict_fn_for_lime(instance):
    prediction = ensemble_predict(instance)
    return prediction

In [40]:
np.random.seed(1)
i = 3
original_row = X_test_corrected.iloc[i]
expl = explainer.explain_instance(
    data_row = X_test_final.iloc[i], 
    predict_fn = predict_fn_for_lime, 
    num_features = 5
)

expl.show_in_notebook(show_all=False)

# Save the LIME explanation to an HTML file
output_dir = 'figures/lime'
html_path = os.path.join(output_dir, "lime_output_kernel_change.html")
expl.save_to_file(html_path)


Intercept 191598.18805928846
Prediction_local [191599.94713157]
Right: 191600.79459261912
